In [1]:
import gurobipy as grb

In [2]:
import random

In [3]:
# declare n , m here

# Number of students
n = 200

# Number of projects
m = 40

In [4]:
# Weights in the objective function

alpha = 0.5
beta = 0.5

In [5]:
def create_friend_groups(n,m):
    G = []
    for a1 in range(n):
        temp = [0]*n
        for a2 in range(n):
            if(a1//5 == a2//5):
                temp[a2] = 1
        G.append(temp)
    return G

def friend_groups_have_same_project(n,m):
  H = []
  for a in range(n):
    temp = [0]*m
    temp[a//5] = 1
    H.append(temp)
  return H

In [6]:
# Generate the social network matrix

G = []
for i in range(n):
  randomlist = []
  for j in range(n):
    if i==j:
      randomlist.append(0)
    else:
      randomlist.append(random.choice([-1,0,1]))
  G.append(randomlist)
# G = create_friend_groups(n,m)

In [7]:
# Generate the project preference matrix

H = []
for i in range(n):
  randomlist = []
  for j in range(m):
    rand_int = random.randint(0,1)
    randomlist.append(rand_int)
  H.append(randomlist)
# H = friend_groups_have_same_project(n, m)

In [8]:
# Generate the project list input

# P = []

# while(1):
#   P = []
#   temp2 = 0
#   for i in range(m):
#     # t1 is the upper bound for the number of teams that can work on that project
#     # t3 is the upper bound for the team size of any of those teams
#     (t1,t3) = (random.randint(1,3), random.randint(3,6))
#     temp2 += t1*t3
#     P.append((t1,t3))
#   if(temp2 >= n):
#     break

P = [(1,5)]*m

# P = [(1,2), (1,3), (1,3)]

print(P)

[(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]


In [9]:
GRB = grb.GRB
# import pandas as pd
# Expert_Skill={

#  (0,"AI"):1,(0,"DB"):1,(0,"ML"):0,(0,"Python"):0,(0,"IR"):0,
#  (1,"AI"):1,(1,"DB"):1,(1,"ML"):1,(1,"Python"):0,(1,"IR"):0,
#  (2,"AI"):1,(2,"DB"):1,(2,"ML"):1,(2,"Python"):0,(2,"IR"):1,
#  (3,"AI"):1,(3,"DB"):0,(3,"ML"):1,(3,"Python"):0,(3,"IR"):1,
#  }
# expert_costs={0:150,1:120,2:120,3:110,4:200}
# expert_capacity={0:3,1:3,2:3,3:3,4:4}
# Project_Skills={
# 1:{"AI":2,"DB":2,"ML":2},2:{"ML":1,"IR":1},3:{"IR":1,"AI":1} }
# Project_Profit={1:120,2:110,3:150}
# projects=[1,2,3]
# Expert_Communication_Cost=[
#  [0,5,2,2],
#  [5,0,5,2],
#  [2,5,0,5],
#  [2,2,5,0]
#  ]
# Experts=[0,1,2,3]
# Skills=[ "AI","DB","ML","IR","Python"]

In [10]:

model =grb.Model("Cluster Assignment")

Academic license - for non-commercial use only - expires 2022-01-06
Using license file /Library/gurobi912/gurobi.lic


In [11]:
y_indices = []
for i in range(m):
    for t in range(P[i][0]):
        for a in range(n):
            y_indices.append((i,t,a))

# print(y_indices)
print(len(y_indices))

8000


In [12]:
y = model.addVars(y_indices,vtype=GRB.BINARY,name="y")
print(len(y))

8000


In [13]:
# l_indices = []
# for i in range(m):
#     for t in range(P[i][0]):
#         for a1 in range(n):
#             for a2 in range(n):
#                 l_indices.append((i,t,a1,a2))

# # print(l_indices)
# print(len(l_indices))

In [14]:
# l = model.addVars(l_indices,vtype=GRB.BINARY,name="l")
# print(len(l))

In [15]:
# for a in range(n):
#     temp = 0
#     for i in range(m):
#         for t in range(P[i][0]):
#             temp += y[(i,t,a)]
for a in range(n):
    model.addConstr( 1 == sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )
    #model.addConstr( 0 >= sum(y[(i,t,a)] for i in range (m) for t in range (P[i][0])) )
    

In [16]:
# for i in range(m):
#     for t in range (P[i][0]):
#         for a1 in range(n):
#             for a2 in range(n):
#                 model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a1)])
#                 model.addConstr(l[(i,t,a1,a2)]<=y[(i,t,a2)])
#                 model.addConstr(l[(i,t,a1,a2)]>= y[(i,t,a1)]+y[(i,t,a2)]-1)

In [17]:
for i in range(m):
    for t in range (P[i][0]):
        model.addConstr( P[i][1] >= sum(y[(i,t,a)] for a in range(n)))

In [18]:
model.setObjective(sum((y[(i,t,a)]*H[a][i]) for i in range(m) for t in range (P[i][0]) for a in range (n)) + 
sum(((y[(i,t,a1)]*y[(i,t,a2)])*G[a1][a2]) for i in range(m) for t in range (P[i][0]) for a1 in range (n) for a2 in range(n) if (a1!=a2))
,GRB.MAXIMIZE)

In [19]:
model.setParam("Timelimit",900)
# model.setParam("NodeMethod",2)
# model.setParam("Method",5)

Changed value of parameter Timelimit to 900.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [20]:

model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 240 rows, 8000 columns and 16000 nonzeros
Model fingerprint: 0x08d27807
Model has 529240 quadratic objective terms
Variable types: 0 continuous, 8000 integer (8000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 103.0000000
Presolve time: 0.77s
Presolved: 240 rows, 8000 columns, 16000 nonzeros
Presolved model has 537240 quadratic objective terms
Variable types: 0 continuous, 8000 integer (8000 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    2377    3.1498764e+03   0.000000e+00   6.769419e+02      5s
    3019    3.1850139e+03   0.000000e+00   3.607497e+02     11s
    3271    3.1938736e+03   0.000000e+

In [21]:

print(model.status , model.objVal)

11 103.0
